# Experiments on 2D Cellular Automata with PyTorch

In [17]:
# Base Data Science snippet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from tqdm import tqdm_notebook

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import sys
sys.path.append("../")

from abio.cellular_automata.dim2 import GameOfLife
from abio.animation import to_rgb_cmap

# Abio Library

In [27]:
%%time
ca = GameOfLife()
states = ca.run(1000,p_init = 0.2,init_size = 200)
states.show_in_notebook()

interactive(children=(Play(value=0, description='Press play', interval=200, max=1000), Output()), _dom_classes…

IntSlider(value=0, max=1000)

Wall time: 1.78 s


In [5]:
states = ca.run(1000,p_init = 0.2,init_size = 50)
states.show_in_notebook()

interactive(children=(Play(value=0, description='Press play', interval=200, max=1000), Output()), _dom_classes…

IntSlider(value=0, max=1000)

In [26]:
states.transform(only_alive = False,resize = (500,500),method = "nearest")
states.show_in_notebook()

TypeError: 'numpy.float32' object cannot be interpreted as an integer

In [59]:
states.save_video(fps = 10)

... Saving video at 2020-10-28_03-37-46_fps10_GameOfLife.mp4


In [52]:
states.save_gif(fps = 25)

... Saving gif at 2020-10-28_03-35-27_fps25_GameOfLife.gif
